In [20]:
import pandas as pd

file_path = r"C:\Users\User\OneDrive\桌面\My Course\碩一下\給台大資料\Data1-unfiltered.xlsx"  


data1_df = pd.read_excel(file_path)  
data1_df = data1_df[data1_df["車種"].str.contains("1.9T|電動車", na=False) | (data1_df["車種"] == "機車")]
data1_df["三邊加總 (CM)"] = data1_df["長"] + data1_df["寬"] + data1_df["高"]

file_path2 = r"C:\Users\User\OneDrive\桌面\My Course\碩一下\給台大資料\Data2.xlsx" 

data2_df = pd.read_excel(file_path2)
data2_df["購車金額"] = data2_df["購車金額"] / 5 / 12
data2_df.rename(columns={"購車金額": "月折舊費用 (Monthly Depreciation)"}, inplace=True)

print(data1_df.head())
print(data2_df.head())


           配送單號  作業內容                作業時間 作業人員        營業所  路線代碼 當配/一般     長  \
0  300308797073   已取貨 2024-12-14 08:39:30  人派1        桃衛1  NS16    一般  37.0   
1  300308797073   分貨中 2024-12-14 15:32:54  林宜蒨    分轉站-NS2  NS16    一般  37.0   
2  300308797073  未知作業 2024-12-15 07:51:47  李育緯  分轉站-NS2奕展  NS16    一般  37.0   
3  300308797073  未知作業 2024-12-15 08:28:45  李育緯  分轉站-NS2奕展  NS16    一般  37.0   
4  300308797073   已取貨 2024-12-15 09:00:26  張凱媛       北衛16  NS16    一般  37.0   

      寬     高   箱號   箱型      車種   重量  
0  28.0  22.0  NaN  1.0  1.9T低溫  1.0  
1  28.0  22.0  NaN  1.0  1.9T低溫  1.0  
2  28.0  22.0  NaN  1.0  8.5T轉運  1.0  
3  28.0  22.0  NaN  1.0  8.5T轉運  1.0  
4  28.0  22.0  NaN  1.0   電動車常溫  1.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311644 entries, 0 to 311643
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   配送單號    311644 non-null  int64         
 1   作業內容    311644 non-null  object        


In [41]:
# Step 1: Drop duplicates by 配送單號 + 車種
unique_deliveries = data1_df.drop_duplicates(subset=["配送單號", "車種"])

# Step 2: Apply volume constraints on deduplicated data
mask_motorcycle = unique_deliveries["三邊加總 (CM)"] < 90
mask_e_motorcycle = unique_deliveries["三邊加總 (CM)"] < 120

# Step 3: Count total eligible deliveries
total_deliveries = len(unique_deliveries)
motorcycle_eligible = unique_deliveries[mask_motorcycle].shape[0]
e_motorcycle_eligible = unique_deliveries[mask_e_motorcycle].shape[0]
truck_only = total_deliveries - e_motorcycle_eligible

# Step 4: Print results
print(f"Total deliveries: {total_deliveries}")
print(f"機車 eligible deliveries (CM < 90): {motorcycle_eligible}")
print(f"電動機車 eligible deliveries (CM < 120): {e_motorcycle_eligible}")
print(f"1.9T only deliveries (CM ≥ 120): {truck_only}")


Total deliveries: 138807
機車 eligible deliveries (CM < 90): 102210
電動機車 eligible deliveries (CM < 120): 122918
1.9T only deliveries (CM ≥ 120): 15889


In [42]:
# Step 1: Drop duplicates by 配送單號 + 車種
unique_deliveries = data1_df.drop_duplicates(subset=["配送單號", "車種"])

# Step 2: Count how many deliveries each 車種 handled
delivery_counts = unique_deliveries["車種"].value_counts().reset_index()
delivery_counts.columns = ["車種", "total_deliveries"]

# Step 3: Estimate average deliveries per month per vehicle type
delivery_counts["estimated_deliveries_per_month"] = delivery_counts["total_deliveries"] / 2.5

# Step 1: Replace 車種 labels to unify 1.9T types
delivery_counts["車種"] = delivery_counts["車種"].replace({"1.9T低溫": "1.9T", "1.9T常溫": "1.9T"})

# Step 2: Group by the unified 車種 and sum the values
combined = delivery_counts.groupby("車種", as_index=False).sum()

# Step 3: Print the combined result
print(combined)


      車種  total_deliveries  estimated_deliveries_per_month
0   1.9T             46845                         18738.0
1     機車             58460                         23384.0
2  電動車常溫             33502                         13400.8


In [43]:
import pulp

# Step1 : Define the LP minimization problem
model = pulp.LpProblem("Vehicle_Import_Optimization", pulp.LpMinimize)

# Step2 : Decision variables: number of vehicles to import (must be integers)
y_1_9t = pulp.LpVariable("y_1.9T", lowBound=0, cat="Integer")
y_emoto = pulp.LpVariable("y_電動機車", lowBound=0, cat="Integer")
y_moto = pulp.LpVariable("y_機車", lowBound=0, cat="Integer")

#Step3-1 : add constraints
model += (
    2.5 * (
        18738 * y_1_9t +
        13400.8 * y_emoto +
        23384 * y_moto
    ) >= 138807,
    "Total_Unique_Deliveries"
)


#Step3-2 : add constraints
model += 2.5 * 18738 * y_1_9t >= 15889, "Large_Item_Only_By_1.9T"

#Step4 : objective function
model += (
    y_1_9t * (9500 + 37000 + 18738 * (2.97 + 12.6)) +
    y_emoto * (8274.92 + 31000 + 13400.8 * (0.37 + 12.2)) +
    y_moto * (1475 + 31000 + 23384 * (0.6 + 12.2))
), "Total_Monthly_Cost"


In [44]:
# Solve the optimization model
model.solve()

# Print the solution status
print("Status:", pulp.LpStatus[model.status])

# Print the number of vehicles to import
print("Optimal number of 1.9T trucks:", y_1_9t.varValue)
print("Optimal number of electric motorcycles:", y_emoto.varValue)
print("Optimal number of motorcycles:", y_moto.varValue)

# Print the minimized total monthly cost
print("Total Monthly Cost (estimated):", pulp.value(model.objective))


Status: Optimal
Optimal number of 1.9T trucks: 1.0
Optimal number of electric motorcycles: 1.0
Optimal number of motorcycles: 1.0
Total Monthly Cost (estimated): 877763.8359999999
